**Problem Statement**: This notebook provides the details about Machine learning predictive model for the problem “Bank loan default risk”.

# **1. importing required libraries and reading the data**


In [ ]:
# Code to mount google drive   (loading the data from your google drive)
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
# Importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import VarianceThreshold      # for Feature selection
from sklearn.feature_selection import mutual_info_classif    # Feature selection - mutual information
from sklearn.model_selection import train_test_split
from imblearn.combine import SMOTETomek
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from collections import Counter
from sklearn.ensemble import IsolationForest



# **Reading data from pickle file**

In [ ]:
import pickle
import joblib 
import pandas as pd

# joblib.dump(FinalXGboostModel, r'/gdrive/MyDrive/PGD_project/FinalXGboostModel.pkl') 
# model = joblib.load(r'/gdrive/MyDrive/PGD_project/FinalXGboostModel.pkl') 
# model.predict(X_test)

DataForML_t=pd.read_pickle(r'/gdrive/MyDrive/PGD_project/DataForML_t.pkl')

In [ ]:
DataForML_t.head()

,TARGET,AGE,AMT_CREDIT,AMT_INCOME_TOTAL,CNT_FAM_MEMBERS,YEARS_EMPLOYED,AMT_ANNUITY,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_DOCUMENT_3,FLAG_DOCUMENT_6,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,EXT_SOURCE_2,AMT_REQ_CREDIT_BUREAU_YEAR,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,CODE_GENDER,FLAG_OWN_CAR,NAME_INCOME_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,NAME_EDUCATION_TYPE,NAME_CONTRACT_TYPE,FLAG_OWN_REALTY,NAME_TYPE_SUITE,ORGANIZATION_TYPE
0,1,25.0,406597.5,202500.0,1.0,1.0,24700.5,1,0,1,1,0,2,2,0,0,0.262949,1,3648.0,2120,2,2.0,Male,N,Working,Single / not married,House / apartment,Secondary / secondary special,Cash loans,Y,Unaccompanied,Business Entity Type 3
1,0,45.0,1293502.5,270000.0,2.0,3.0,35698.5,1,0,1,1,0,1,1,0,0,0.622246,0,1186.0,291,1,0.0,Female,N,State servant,Married,House / apartment,Higher education,Cash loans,N,Family,School
2,0,52.0,135000.0,67500.0,1.0,0.0,6750.0,1,1,1,0,0,2,2,0,0,0.555912,0,4260.0,2531,0,0.0,Male,Y,Working,Single / not married,House / apartment,Secondary / secondary special,Revolving loans,Y,Unaccompanied,Government
3,0,52.0,312682.5,135000.0,2.0,8.0,29686.5,1,0,0,1,0,2,2,0,0,0.650442,0,9833.0,2437,2,0.0,Female,N,Working,Civil marriage,House / apartment,Secondary / secondary special,Cash loans,Y,Unaccompanied,Business Entity Type 3
4,0,54.0,513000.0,121500.0,1.0,8.0,21865.5,1,0,0,0,0,2,2,0,1,0.322738,0,4311.0,3458,0,0.0,Male,N,Working,Single / not married,House / apartment,Secondary / secondary special,Cash loans,Y,Unaccompanied,Religion


In [ ]:
DataForML_t['CODE_GENDER'].replace({'Male':1, 'Female':0}, inplace=True)
DataForML_t['FLAG_OWN_CAR'].replace({'Y':1, 'N':0}, inplace=True)
DataForML_t['NAME_CONTRACT_TYPE'].replace({'Revolving loans':1, 'Cash loans':0}, inplace=True)
DataForML_t['FLAG_OWN_REALTY'].replace({'Y':1, 'N':0}, inplace=True)

In [ ]:
# Treating all the nominal variables at once using dummy variables
DataForML_Numeric=pd.get_dummies(DataForML_t)
DataForML_Numeric.head()

,TARGET,AGE,AMT_CREDIT,AMT_INCOME_TOTAL,CNT_FAM_MEMBERS,YEARS_EMPLOYED,AMT_ANNUITY,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_DOCUMENT_3,FLAG_DOCUMENT_6,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,EXT_SOURCE_2,AMT_REQ_CREDIT_BUREAU_YEAR,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,CODE_GENDER,FLAG_OWN_CAR,NAME_CONTRACT_TYPE,FLAG_OWN_REALTY,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,...,ORGANIZATION_TYPE_Industry: type 13,ORGANIZATION_TYPE_Industry: type 2,ORGANIZATION_TYPE_Industry: type 3,ORGANIZATION_TYPE_Industry: type 4,ORGANIZATION_TYPE_Industry: type 5,ORGANIZATION_TYPE_Industry: type 6,ORGANIZATION_TYPE_Industry: type 7,ORGANIZATION_TYPE_Industry: type 8,ORGANIZATION_TYPE_Industry: type 9,ORGANIZATION_TYPE_Insurance,ORGANIZATION_TYPE_Kindergarten,ORGANIZATION_TYPE_Legal Services,ORGANIZATION_TYPE_Medicine,ORGANIZATION_TYPE_Military,ORGANIZATION_TYPE_Mobile,ORGANIZATION_TYPE_Other,ORGANIZATION_TYPE_Police,ORGANIZATION_TYPE_Postal,ORGANIZATION_TYPE_Realtor,ORGANIZATION_TYPE_Religion,ORGANIZATION_TYPE_Restaurant,ORGANIZATION_TYPE_School,ORGANIZATION_TYPE_Security,ORGANIZATION_TYPE_Security Ministries,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Services,ORGANIZATION_TYPE_Telecom,ORGANIZATION_TYPE_Trade: type 1,ORGANIZATION_TYPE_Trade: type 2,ORGANIZATION_TYPE_Trade: type 3,ORGANIZATION_TYPE_Trade: type 4,ORGANIZATION_TYPE_Trade: type 5,ORGANIZATION_TYPE_Trade: type 6,ORGANIZATION_TYPE_Trade: type 7,ORGANIZATION_TYPE_Transport: type 1,ORGANIZATION_TYPE_Transport: type 2,ORGANIZATION_TYPE_Transport: type 3,ORGANIZATION_TYPE_Transport: type 4,ORGANIZATION_TYPE_University,ORGANIZATION_TYPE_XNA
0,1,25.0,406597.5,202500.0,1.0,1.0,24700.5,1,0,1,1,0,2,2,0,0,0.262949,1,3648.0,2120,2,2.0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,45.0,1293502.5,270000.0,2.0,3.0,35698.5,1,0,1,1,0,1,1,0,0,0.622246,0,1186.0,291,1,0.0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,52.0,135000.0,67500.0,1.0,0.0,6750.0,1,1,1,0,0,2,2,0,0,0.555912,0,4260.0,2531,0,0.0,1,1,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,52.0,312682.5,135000.0,2.0,8.0,29686.5,1,0,0,1,0,2,2,0,0,0.650442,0,9833.0,2437,2,0.0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,54.0,513000.0,121500.0,1.0,8.0,21865.5,1,0,0,0,0,2,2,0,1,0.322738,0,4311.0,3458,0,0.0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
print(DataForML_Numeric.shape)

(306486, 116)


# **Machine Learning**



In [ ]:
# Printing all the column names for our reference
print(list(DataForML_Numeric.columns))

['TARGET', 'AGE', 'AMT_CREDIT', 'AMT_INCOME_TOTAL', 'CNT_FAM_MEMBERS', 'YEARS_EMPLOYED', 'AMT_ANNUITY', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_PHONE', 'FLAG_DOCUMENT_3', 'FLAG_DOCUMENT_6', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'EXT_SOURCE_2', 'AMT_REQ_CREDIT_BUREAU_YEAR', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'NAME_CONTRACT_TYPE', 'FLAG_OWN_REALTY', 'NAME_INCOME_TYPE_Businessman', 'NAME_INCOME_TYPE_Commercial associate', 'NAME_INCOME_TYPE_Maternity leave', 'NAME_INCOME_TYPE_Pensioner', 'NAME_INCOME_TYPE_State servant', 'NAME_INCOME_TYPE_Student', 'NAME_INCOME_TYPE_Unemployed', 'NAME_INCOME_TYPE_Working', 'NAME_FAMILY_STATUS_Civil marriage', 'NAME_FAMILY_STATUS_Married', 'NAME_FAMILY_STATUS_Separated', 'NAME_FAMILY_STATUS_Single / not married', 'NAME_FAMILY_STATUS_Unknown', 'NAME_FAMILY_STATUS_Widow', 'NAME_HOUSING_TYPE_Co-op a

In [ ]:
# Separate Target Variable and Predictor Variables
TargetVariable='TARGET'
Predictors=[ 'AGE', 'AMT_CREDIT', 'AMT_INCOME_TOTAL', 'CNT_FAM_MEMBERS', 'YEARS_EMPLOYED', 'AMT_ANNUITY', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_PHONE', 
             'FLAG_DOCUMENT_3', 'FLAG_DOCUMENT_6', 'REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 
             'EXT_SOURCE_2', 'AMT_REQ_CREDIT_BUREAU_YEAR', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OBS_30_CNT_SOCIAL_CIRCLE', 'DEF_30_CNT_SOCIAL_CIRCLE', 
             'NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR',  'FLAG_OWN_REALTY',
             
             'NAME_TYPE_SUITE_Unaccompanied', 'NAME_TYPE_SUITE_Family', 'NAME_TYPE_SUITE_Spouse, partner', 'NAME_TYPE_SUITE_Children', 
             'NAME_TYPE_SUITE_Other_B', 'NAME_TYPE_SUITE_Other_A', 'NAME_TYPE_SUITE_Group of people',
             
             'NAME_EDUCATION_TYPE_Secondary / secondary special', 'NAME_EDUCATION_TYPE_Higher education', 'NAME_EDUCATION_TYPE_Incomplete higher',
             'NAME_EDUCATION_TYPE_Lower secondary', 'NAME_EDUCATION_TYPE_Academic degree',  
               
             'NAME_FAMILY_STATUS_Married', 'NAME_FAMILY_STATUS_Single / not married', 'NAME_FAMILY_STATUS_Civil marriage', 
             'NAME_FAMILY_STATUS_Separated', 'NAME_FAMILY_STATUS_Widow', 'NAME_FAMILY_STATUS_Unknown',     

             'NAME_HOUSING_TYPE_House / apartment', 'NAME_HOUSING_TYPE_With parents', 'NAME_HOUSING_TYPE_Municipal apartment',  
             'NAME_HOUSING_TYPE_Rented apartment', 'NAME_HOUSING_TYPE_Office apartment', 'NAME_HOUSING_TYPE_Co-op apartment',         

             'ORGANIZATION_TYPE_Business Entity Type 3', 'ORGANIZATION_TYPE_XNA', 'ORGANIZATION_TYPE_Self-employed', 'ORGANIZATION_TYPE_Other',
             'ORGANIZATION_TYPE_Medicine', 'ORGANIZATION_TYPE_Business Entity Type 2', 'ORGANIZATION_TYPE_Government', 'ORGANIZATION_TYPE_School',
             'ORGANIZATION_TYPE_Trade: type 7', 'ORGANIZATION_TYPE_Kindergarten', 'ORGANIZATION_TYPE_Construction', 'ORGANIZATION_TYPE_Business Entity Type 1',
             'ORGANIZATION_TYPE_Transport: type 4', 'ORGANIZATION_TYPE_Trade: type 3', 'ORGANIZATION_TYPE_Industry: type 9', 'ORGANIZATION_TYPE_Industry: type 3',
             'ORGANIZATION_TYPE_Security', 'ORGANIZATION_TYPE_Housing', 'ORGANIZATION_TYPE_Industry: type 11', 'ORGANIZATION_TYPE_Military',
             'ORGANIZATION_TYPE_Bank', 'ORGANIZATION_TYPE_Agriculture', 'ORGANIZATION_TYPE_Police', 'ORGANIZATION_TYPE_Transport: type 2',
             'ORGANIZATION_TYPE_Postal', 'ORGANIZATION_TYPE_Security Ministries', 'ORGANIZATION_TYPE_Trade: type 2', 'ORGANIZATION_TYPE_Restaurant',
             'ORGANIZATION_TYPE_Services', 'ORGANIZATION_TYPE_University', 'ORGANIZATION_TYPE_Industry: type 7', 'ORGANIZATION_TYPE_Transport: type 3',
             'ORGANIZATION_TYPE_Industry: type 1', 'ORGANIZATION_TYPE_Hotel', 'ORGANIZATION_TYPE_Electricity', 'ORGANIZATION_TYPE_Industry: type 4',
             'ORGANIZATION_TYPE_Trade: type 6', 'ORGANIZATION_TYPE_Industry: type 5', 'ORGANIZATION_TYPE_Insurance', 'ORGANIZATION_TYPE_Telecom',
             'ORGANIZATION_TYPE_Emergency', 'ORGANIZATION_TYPE_Industry: type 2', 'ORGANIZATION_TYPE_Advertising', 'ORGANIZATION_TYPE_Realtor',
             'ORGANIZATION_TYPE_Culture', 'ORGANIZATION_TYPE_Industry: type 12', 'ORGANIZATION_TYPE_Trade: type 1',
             'ORGANIZATION_TYPE_Mobile', 'ORGANIZATION_TYPE_Legal Services', 'ORGANIZATION_TYPE_Cleaning', 'ORGANIZATION_TYPE_Transport: type 1', 
             'ORGANIZATION_TYPE_Industry: type 6', 'ORGANIZATION_TYPE_Industry: type 10', 'ORGANIZATION_TYPE_Religion', 'ORGANIZATION_TYPE_Industry: type 13',
             'ORGANIZATION_TYPE_Trade: type 4', 'ORGANIZATION_TYPE_Trade: type 5', 'ORGANIZATION_TYPE_Industry: type 8',    
                  
             'NAME_INCOME_TYPE_Working', 'NAME_INCOME_TYPE_Commercial associate', 'NAME_INCOME_TYPE_Pensioner', 'NAME_INCOME_TYPE_State servant', 
             'NAME_INCOME_TYPE_Unemployed', 'NAME_INCOME_TYPE_Student', 'NAME_INCOME_TYPE_Businessman', 'NAME_INCOME_TYPE_Maternity leave']

X_ml=DataForML_Numeric[Predictors].values
y_ml=DataForML_Numeric[TargetVariable].values

# Split the data into training and testing set
from sklearn.model_selection import train_test_split
X_train_ml, X_test_ml, y_train_ml, y_test_ml = train_test_split(X_ml, y_ml, test_size=0.3, random_state=428)


# **Standardization/Normalization of data**

In [ ]:
### Sandardization of data ###
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# Choose either standardization or Normalization
# On this data Min Max Normalization produced better results

# Choose between standardization and MinMAx normalization
#PredictorScaler=StandardScaler()
PredictorScaler=MinMaxScaler()

# Storing the fit object for later reference
PredictorScalerFit=PredictorScaler.fit(X_ml)

# Generating the standardized values of X
X_ml=PredictorScalerFit.transform(X_ml)

# Split the data into training and testing set
from sklearn.model_selection import train_test_split
X_train_ml, X_test_ml, y_train_ml, y_test_ml = train_test_split(X_ml, y_ml, test_size=0.3, random_state=42)

In [ ]:
# Sanity check for the sampled data
print(X_train_ml.shape)
print(y_train_ml.shape)
print(X_test_ml.shape)
print(y_test_ml.shape)

(214540, 115)
(214540,)
(91946, 115)
(91946,)


In [ ]:
# Get the default and non-default

# default_1     = y_test[y_test['TARGET']==1]
# non_default_1 = y_test[y_test['TARGET']==0]

default_1     = y_test_ml[y_test_ml==1]
non_default_1 = y_test_ml[y_test_ml==0]

print(default_1.shape,non_default_1.shape)

(7437,) (84509,)


# **Oversampling**

In [ ]:
from imblearn.over_sampling import RandomOverSampler

In [ ]:
os = RandomOverSampler(0.5)
X_train_mlos, y_train_mlos = os.fit_resample(X_train_ml, y_train_ml)

print("The number of classes before fit {}".format(Counter(y_train_ml)))
print("The number of classes after fit {}".format(Counter(y_train_mlos)))

/usr/local/lib/python3.7/dist-packages/imblearn/utils/_validation.py:591: FutureWarning: Pass sampling_strategy=0.5 as keyword args. From version 0.9 passing these as positional arguments will result in an error
  FutureWarning,


The number of classes before fit Counter({0: 197188, 1: 17352})
The number of classes after fit Counter({0: 197188, 1: 98594})


In [ ]:
X_train_mlos.shape

(295782, 115)

In [ ]:
X_train_ml.shape

(214540, 115)

# **Anamoly Detection and Removal**

In [ ]:
# identify outliers in the training dataset
iso = IsolationForest(contamination=0.1)
y_pred = iso.fit_predict(X_train_mlos)

In [ ]:
y_pred

array([ 1,  1,  1, ..., -1,  1,  1])

In [ ]:
# function to get unique values
def unique(list1):
    y_pred_unique = np.array(list1)
    print(np.unique(y_pred_unique))

In [ ]:
print("the unique values from 1st list is")
unique(y_pred)

the unique values from 1st list is
[-1  1]


In [ ]:
# y_train_mlos.head()

In [ ]:
# select all rows that are not outliers
mask = y_pred != -1
X_train_iso, y_train_iso = X_train_mlos[mask], y_train_mlos[mask]

In [ ]:
X_train_mlos.shape, y_train_mlos.shape

((295782, 115), (295782,))

In [ ]:
X_train_iso.shape, y_train_iso.shape

((266203, 115), (266203,))

In [ ]:
print("The number of classes after outlier removal {}".format(Counter(y_train_iso)))

The number of classes after outlier removal Counter({0: 177191, 1: 89012})


# **Stacking**

In [ ]:
# make a prediction with a stacking ensemble
from sklearn.datasets import make_classification
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB

Trained without Outlier removal

In [ ]:
# define the base models
level0 = list()
level0.append(('lr', LogisticRegression(C=100,penalty='l2', solver='liblinear')))
level0.append(('cart', DecisionTreeClassifier(max_depth=7,criterion='entropy')))
level0.append(('bayes', GaussianNB()))


# define meta learner model
level1 = LogisticRegression()

# define the stacking ensemble
model = StackingClassifier(estimators=level0, final_estimator=level1)
# fit the model on all available data
stack_2 = model.fit(X_train_mlos,y_train_mlos)

prediction_stack=stack_2.predict(X_test_ml)

# Measuring accuracy on Testing Data
from sklearn import metrics
print(metrics.classification_report(y_test_ml, prediction_stack))
print(metrics.confusion_matrix(y_test_ml, prediction_stack))

# Printing the Overall Accuracy of the model
F1_Score=metrics.f1_score(y_test_ml, prediction_stack, average='weighted')
print('Accuracy of the model on Testing Sample Data:', round(F1_Score,2))

              precision    recall  f1-score   support

           0       0.94      0.90      0.92     84509
           1       0.21      0.30      0.25      7437

    accuracy                           0.85     91946
   macro avg       0.57      0.60      0.58     91946
weighted avg       0.88      0.85      0.86     91946

[[75893  8616]
 [ 5169  2268]]
Accuracy of the model on Testing Sample Data: 0.86


decision tree as meta learener

In [ ]:
# before Iso
# define the base models
level0 = list()
level0.append(('lr', LogisticRegression(C=100,penalty='l2', solver='liblinear')))
level0.append(('cart', DecisionTreeClassifier(max_depth=7,criterion='entropy')))
level0.append(('bayes', GaussianNB()))


# define meta learner model
level1 = DecisionTreeClassifier()

# define the stacking ensemble
model = StackingClassifier(estimators=level0, final_estimator=level1)
# fit the model on all available data
stack_dt = model.fit(X_train_mlos,y_train_mlos)

prediction_stack=stack_dt.predict(X_test_ml)

# Measuring accuracy on Testing Data
from sklearn import metrics
print(metrics.classification_report(y_test_ml, prediction_stack))
print(metrics.confusion_matrix(y_test_ml, prediction_stack))

# Printing the Overall Accuracy of the model
F1_Score=metrics.f1_score(y_test_ml, prediction_stack, average='weighted')
print('Accuracy of the model on Testing Sample Data:', round(F1_Score,2))

              precision    recall  f1-score   support

           0       0.93      0.77      0.84     84509
           1       0.12      0.34      0.17      7437

    accuracy                           0.74     91946
   macro avg       0.52      0.56      0.51     91946
weighted avg       0.86      0.74      0.79     91946

[[65409 19100]
 [ 4937  2500]]
Accuracy of the model on Testing Sample Data: 0.79


Trained with Outlier removal

In [ ]:
# After Iso
# define the base models
level0 = list()
level0.append(('lr', LogisticRegression(C=100,penalty='l2', solver='liblinear')))
level0.append(('cart', DecisionTreeClassifier(max_depth=7,criterion='entropy')))
level0.append(('bayes', GaussianNB()))


# define meta learner model
level1 = LogisticRegression()

# define the stacking ensemble
model = StackingClassifier(estimators=level0, final_estimator=level1)
# fit the model on all available data
stack_2 = model.fit(X_train_iso,y_train_iso)

prediction_stack=stack_2.predict(X_test_ml)

# Measuring accuracy on Testing Data
from sklearn import metrics
print(metrics.classification_report(y_test_ml, prediction_stack))
print(metrics.confusion_matrix(y_test_ml, prediction_stack))

# Printing the Overall Accuracy of the model
F1_Score=metrics.f1_score(y_test_ml, prediction_stack, average='weighted')
print('Accuracy of the model on Testing Sample Data:', round(F1_Score,2))

              precision    recall  f1-score   support

           0       0.94      0.89      0.91     84509
           1       0.21      0.32      0.25      7437

    accuracy                           0.84     91946
   macro avg       0.57      0.61      0.58     91946
weighted avg       0.88      0.84      0.86     91946

[[75245  9264]
 [ 5043  2394]]
Accuracy of the model on Testing Sample Data: 0.86


Trained without Outlier removal (BernoulliNB)

In [ ]:
#before iso
from sklearn.naive_bayes import BernoulliNB
# define the base models
level0 = list()
level0.append(('lr', LogisticRegression(C=100,penalty='l2', solver='liblinear')))
level0.append(('cart', DecisionTreeClassifier(max_depth=7,criterion='entropy')))
level0.append(('bayes', BernoulliNB()))


# define meta learner model
level1 = LogisticRegression()

# define the stacking ensemble
model = StackingClassifier(estimators=level0, final_estimator=level1)
# fit the model on all available data
stack_2 = model.fit(X_train_mlos,y_train_mlos)

prediction_stack=stack_2.predict(X_test_ml)

# Measuring accuracy on Testing Data
from sklearn import metrics
print(metrics.classification_report(y_test_ml, prediction_stack))
print(metrics.confusion_matrix(y_test_ml, prediction_stack))

# Printing the Overall Accuracy of the model
F1_Score=metrics.f1_score(y_test_ml, prediction_stack, average='weighted')
print('Accuracy of the model on Testing Sample Data:', round(F1_Score,2))

              precision    recall  f1-score   support

           0       0.94      0.90      0.92     84509
           1       0.21      0.30      0.25      7437

    accuracy                           0.85     91946
   macro avg       0.57      0.60      0.58     91946
weighted avg       0.88      0.85      0.86     91946

[[75884  8625]
 [ 5173  2264]]
Accuracy of the model on Testing Sample Data: 0.86


Trained with Outlier removal (BernoulliNB)

In [ ]:
#after iso
from sklearn.naive_bayes import BernoulliNB
# define the base models
level0 = list()
level0.append(('lr', LogisticRegression(C=100,penalty='l2', solver='liblinear')))
level0.append(('cart', DecisionTreeClassifier(max_depth=7,criterion='entropy')))
level0.append(('bayes', BernoulliNB()))


# define meta learner model
level1 = LogisticRegression()

# define the stacking ensemble
model = StackingClassifier(estimators=level0, final_estimator=level1)
# fit the model on all available data
stack_2 = model.fit(X_train_iso,y_train_iso)

prediction_stack=stack_2.predict(X_test_ml)

# Measuring accuracy on Testing Data
from sklearn import metrics
print(metrics.classification_report(y_test_ml, prediction_stack))
print(metrics.confusion_matrix(y_test_ml, prediction_stack))

# Printing the Overall Accuracy of the model
F1_Score=metrics.f1_score(y_test_ml, prediction_stack, average='weighted')
print('Accuracy of the model on Testing Sample Data:', round(F1_Score,2))

              precision    recall  f1-score   support

           0       0.94      0.89      0.91     84509
           1       0.21      0.32      0.25      7437

    accuracy                           0.84     91946
   macro avg       0.57      0.61      0.58     91946
weighted avg       0.88      0.84      0.86     91946

[[75260  9249]
 [ 5044  2393]]
Accuracy of the model on Testing Sample Data: 0.86
